# Unidades de Materiais
> Neste notebook baixamos os dados de materiais da api disponibilizada pelo MGI (dados abertos) e criamos uma base.
>
> A documentação da API do MGI pode ser encontrada em https://dadosabertos.compras.gov.br/swagger-ui/index.html#/

In [2]:
import json
import duckdb
import ipywidgets
from fastprogress.fastprogress import master_bar, progress_bar
import pandas as pd
import requests
import os

/Users/fredguth/Code/sus/sqlmesh/.env/lib/python3.11/site-packages/fastprogress/fastprogress.py:107: UserWarning: Couldn't import ipywidgets properly, progress bar will use console behavior
  warn("Couldn't import ipywidgets properly, progress bar will use console behavior")


Agora que já baixamos os Materiais e colocamos no BD, vamos baixar as unidades que são usadas pelos materiais que baixamos.

In [3]:
db = duckdb.connect('./db.db') #copia

In [6]:
pdms = list(db.sql("select distinct codigoPdm from br_mgi__dev.materiais order by codigoPdm").df()['codigoPdm'])
pdms

[13,
 45,
 85,
 265,
 348,
 349,
 350,
 351,
 352,
 353,
 354,
 355,
 356,
 357,
 358,
 359,
 360,
 361,
 362,
 363,
 364,
 365,
 366,
 370,
 371,
 372,
 373,
 374,
 375,
 377,
 378,
 379,
 380,
 381,
 382,
 383,
 384,
 385,
 386,
 387,
 388,
 389,
 390,
 391,
 392,
 393,
 394,
 395,
 396,
 397,
 399,
 400,
 401,
 402,
 403,
 404,
 405,
 406,
 407,
 408,
 409,
 410,
 418,
 420,
 421,
 422,
 423,
 424,
 425,
 426,
 427,
 428,
 429,
 430,
 431,
 432,
 433,
 434,
 435,
 436,
 437,
 438,
 439,
 440,
 441,
 442,
 443,
 444,
 445,
 446,
 447,
 448,
 449,
 450,
 451,
 452,
 453,
 454,
 455,
 458,
 459,
 460,
 461,
 462,
 463,
 464,
 465,
 467,
 468,
 474,
 475,
 498,
 515,
 516,
 517,
 518,
 525,
 526,
 527,
 529,
 530,
 531,
 532,
 533,
 534,
 535,
 539,
 540,
 541,
 542,
 543,
 545,
 546,
 547,
 549,
 561,
 571,
 572,
 573,
 574,
 575,
 576,
 709,
 711,
 712,
 713,
 714,
 716,
 717,
 719,
 720,
 721,
 723,
 724,
 725,
 729,
 730,
 731,
 732,
 733,
 734,
 735,
 736,
 737,
 738,
 740,
 741,
 

In [9]:
def fetch_and_save(codes):
    base_url = 'https://dadosabertos.compras.gov.br/modulo-material/6_consultarMaterialUnidadeFornecimento'
    
    def save_json(code, page):
        response = requests.get(f'{base_url}?codigoPdm={code}&pagina={page}').json()
        with open(f'../seeds/materiais/unidades/codigoPdm={code}/page_{page}.json', 'w') as f:
            json.dump(response['resultado'], f)
        return response
    
    mb = master_bar(codes)
    for code in mb:
        os.makedirs(f'../seeds/materiais/unidades/codigoPdm={code}', exist_ok=True)
        response = save_json(code, 1)
        total_pages = response['totalPaginas']
        pb = progress_bar(range(2, total_pages + 1), parent=mb, leave=False)
        for page in pb:
            save_json(code, page)

In [11]:
fetch_and_save(pdms)